In [1]:
import pandas as pd
import rdflib
from rdflib.namespace import RDF
from rdflib import Namespace
from rdflib import URIRef, Literal

In [2]:
global g1
g1=rdflib.Graph()
g1.parse('C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/Graph_metafile.nt',format='nt')
print(len(g1))

582


In [3]:
#************Change Path to files************
union_data=pd.read_csv("C:/Users/SESA456791/Desktop/Ontology-Bh/Files/unionupd.csv", header=None) 

In [4]:
union_data.columns=['Cat1','Cat2','Org1','Org2']

In [5]:
#Fucntion:Changing plural to singular
def change_cat(col_nam,df):
    singular_plural_data=pd.read_csv("C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Singular_Plural.csv") 
    for i in range(singular_plural_data.shape[0]):
        val=singular_plural_data.iloc[i,0]
        new_val=singular_plural_data.iloc[i,1]
        df.loc[(df[col_nam]).apply(lambda x: x.strip())==val,col_nam]=new_val
    return(df)

In [6]:
#Fucntion Call:Changing plural to singular
union_data=change_cat('Cat1',union_data)
union_data=change_cat('Cat2',union_data)

In [7]:
union_data.head(20)

,Cat1,Cat2,Org1,Org2
0,Union Government,Apex Body,"['President of India', 'http://presidentofindi...","['Presidents Secretariat', 'http://rashtrapati..."
1,Union Government,Apex Body,"['President of India', 'http://presidentofindi...","['Presidents Secretariat Helpline', 'http://he..."
2,Union Government,Apex Body,"['Vice President of India', 'http://vicepresid...",[]
3,Union Government,Apex Body,"[""Prime Minister's Office"", 'http://pmindia.go...","['Direct Benefit Transfer (DBT), MIS - DBT Pla..."
4,Union Government,Apex Body,"[""Prime Minister's Office"", 'http://pmindia.go...",['Economic Advisory Council to The Prime Minis...
5,Union Government,Apex Body,"[""Prime Minister's Office"", 'http://pmindia.go...","['India Innovation Portal (IIP)', 'http://www...."
6,Union Government,Apex Body,"[""Prime Minister's Office"", 'http://pmindia.go...","[""Mann ki Baat by Hon'ble Prime Minister of In..."
7,Union Government,Apex Body,"[""Prime Minister's Office"", 'http://pmindia.go...",['National Critical Information Infrastructure...
8,Union Government,Apex Body,"[""Prime Minister's Office"", 'http://pmindia.go...","['National Innovation Council', 'http://innova..."
9,Union Government,Apex Body,"[""Prime Minister's Office"", 'http://pmindia.go...","['National Rainfed Area Authority (NRAA)', 'ht..."


In [8]:
global org
global bhorg
global skos
global bhcatres
global bhorgres

org=Namespace("http://www.w3.org/ns/org#")
bhorg=Namespace("http://semanticwebindia.in/bharathi/ont/org/")
skos=Namespace("http://www.w3.org/2004/02/skos/core#")
bhcatres=Namespace("http://semanticwebindia.in/bharathi/res/cat/")
bhorgres=Namespace("http://semanticwebindia.in/bharathi/res/org/")

In [9]:
pd.set_option('display.max_colwidth', -1)#housekeeping

In [10]:
#finding n_org max
global n_org
l_lit=[]
for s,p,o in g1.triples((None, RDF.type, bhorg.Organisation)):
    str_s_uri=str(s)
    pos=str_s_uri.rfind('/')
    num=int(str_s_uri[pos+1:len(str_s_uri)])
    num_s=str(num)
    num_s=num_s[1:]
    num_i=int(num_s)
    l_lit.append(num_i) #list of n_org identifier values
l_lit.sort() #sorting the list
n_org=l_lit[-1] #the current maximum value of n_org

#No need to find n_cat max, just one skos:category- topOT
global n_cat
n_cat=0
cat_list=[]
for s,p,o in g1.triples((None, skos.prefLabel, None)):
    str_s_uri=str(s)
    pos=str_s_uri.rfind('/')
    if str_s_uri[pos+1:len(str_s_uri)]!='topOT':
        num=int(str_s_uri[pos+1:len(str_s_uri)])
        num_s=str(num)
        num_s=num_s[1:]
        num_i=int(num_s)
        cat_list.append(num_i) #list of n_org identifier values
cat_list.sort() #sorting the list
n_cat=cat_list[-1] #the current maximum value of n_org

In [11]:
#create fetch function for cat_id
def fetch_n_cat(cat):
    global n_cat
    global g1
    cat_id=""
    for s,p,o in g1.triples((None, skos.prefLabel,Literal(cat))):
        cat_id=s
    if cat_id=="":
        bhcatres="http://semanticwebindia.in/bharathi/res/cat/"
        n_cat+=1
        cat_id=URIRef(bhcatres+str('2')+str(n_cat))
        #adding relevant triples
#         g1.add((cat_id,skos.broader, URIRef(bhcatres+'topOT')))
#         g1.add((URIRef(bhcatres+'topOT'),skos.broaderOf,cat_id))
        g1.add((cat_id,RDF.type,skos.Concept))
        g1.add((cat_id,skos.prefLabel, Literal(cat)))
        g1.add((cat_id,skos.altLabel, Literal(cat)))
        g1.add((cat_id,skos.inScheme,URIRef(bhcatres+'OrganisationType')))
    return(cat_id)

#create fetch function for org_id
def fetch_n_org(org_nam,org_url="",org_abb=""):
    global g1
    global n_org
    org_id=""
    for s,p,o in g1.triples((None, bhorg.hasName,Literal(org_nam))):
        org_id=s
    if org_id=="":
        bhorgres="http://semanticwebindia.in/bharathi/res/org/"
        n_org+=1
        org_id=URIRef(bhorgres+str('1')+str(n_org))
        g1.add((org_id, bhorg.hasName, Literal(org_nam)))
        g1.add((org_id, bhorg.hasAlternateName, Literal(org_nam)))
        if org_url!="":
            g1.add((org_id, bhorg.hasURL, Literal(org_url)))
        if org_abb!="":
            g1.add((org_id, bhorg.hasAbbreviation, Literal(org_abb)))
        g1.add((org_id,RDF.type,bhorg.Organisation))
        g1.add((org_id,org.identifier,Literal(n_org)))
        g1.add((org_id,org.hasOrganizationType,URIRef(bhcatres+str('21')))) #skos(Government) :bhcatres/21
    return(org_id)

In [12]:
#create fetch_cat_broader for broader relation
def fetch_cat_broader(cat1_id,cat2_id):
    global g1
    if not (cat2_id, skos.broader,cat1_id) in g1:
        g1.add((cat2_id, skos.broader,cat1_id))
    if not (cat1_id, skos.broaderOf,cat2_id) in g1:
        g1.add((cat1_id, skos.broaderOf,cat2_id))
        
#create fetch_org_subOrg for sub-organization relation
def fetch_org_subOrg(org1_id,org2_id):
    global g1
    if not (org1_id,org.hasSubOrganization,org2_id) in g1:
        g1.add((org1_id,org.hasSubOrganization,org2_id))
    if not (org2_id,org.subOrganizationOf,org1_id) in g1:
        g1.add((org2_id,org.subOrganizationOf,org1_id))

In [13]:
#Clean org1/org2 and return list
#***********Original**************#

# def org_scrub(org_n):
#     org_str_raw=org_n
#     org_str_raw=org_str_raw.replace('"','\'')
#     org_str_raw=org_str_raw.replace("' '","','")
#     org_str_raw=org_str_raw.replace("[","")
#     org_str_raw=org_str_raw.replace("]","")
#     org_str_raw=org_str_raw.strip()
#     l=org_str_raw.split(',')
#     l_scrub=[] #Cleaned list of 
#     if len(l)>0:
#         for x in l:
#             x.strip()
#             if x=='':
#                 continue
#             else:
#                 x=x.replace("'","")
#                 l_scrub.append(x)
#     return(l_scrub)

#**********borrowed from State&UT_v5*************
# def org_scrub(org_n):
#     org_n=org_n.replace('[','')
#     org_n=org_n.replace(']','')
#     ls=list(org_n)
#     l_scrub=[]
#     i=0
#     while(i<len(ls)):
#         if ls[i] in ["\'",'\"']:
#             sch_char=ls[i]
#             #l_scrub=[]
#             for j in range(i+1,len(ls)):
#                 if ls[j]==sch_char:
#                     val=ls[i:j+1]
#                     val_s=""
#                     for x in val:
#                         val_s=val_s+x
#                     val_s=val_s.replace(sch_char,"")
#                     val_s=val_s.strip()
#                     if val_s!=',':
#                         l_scrub.append(val_s)
#                     i=j
#         i=i+1
#     return(l_scrub)

In [14]:
def org_scrub(org_n):
    l_scrub=[]
    val=org_n
    val=str(val)
    val=val.replace('[',"")
    val=val.replace(']',"")
    if len(val)==0:
        return(l_scrub)
    #print(val)
    pos_url=val.find('http')
    #print(pos_url)
    if pos_url!=-1:
        val_1=val[:pos_url]+'|'+val[pos_url:]
        pos_end_url=val_1[pos_url+1:].find("'")
        val_2=val_1[:pos_url+1+pos_end_url]+'|'+val_1[pos_url+1+pos_end_url:]
    else:
        sch_ch=val[0]
        pos_end=val[1:].find(sch_ch)
        #print(val[:pos_end+2])
        val_2=val[:pos_end+2]+"|"+val[pos_end+2:]
        #print(val_2)
    org_ls=val_2.split('|')
    
    for i in range(len(org_ls)):
        ele=org_ls[i]
        ele_ls=list(ele)
        clean_c=1
        while clean_c>0 and len(ele_ls)>0:
            if ele_ls[0] in ["'",'"',',',' '] and len(ele_ls)>0:
                ele_ls.pop(0)
                #print(ele_ls)
            if len(ele_ls)>0:
                if ele_ls[0] in ["'",'"',',',' ']:
                    clean_c=1
                else:
                    clean_c=0
            else:
                clean_c=0
        clean_c=1
        while clean_c>0 and len(ele_ls)>0:
            if ele_ls[-1] in ["'",'"',',',' ']:
                ele_ls.pop(-1)
            if len(ele_ls)>0:
                if ele_ls[-1] in ["'",'"',',',' ']:
                    clean_c=1
                else:
                    clean_c=0
            else:
                clean_c=0
        ele_j=''.join(ele_ls)
        l_scrub.append(ele_j)
    return(l_scrub)

In [15]:
#Iteration over union_data rows
for i in range(union_data.shape[0]):
    
    cat1=union_data.iloc[i,0].strip()
    cat2=union_data.iloc[i,1].strip()
    org1=union_data.iloc[i,2]
    org2=union_data.iloc[i,3]
        
    #check if cat1_id already exists, else create
    cat1_id=fetch_n_cat(cat1)
    
    #check if ((cat1 broader cat21)), ((cat21 broaderOf cat1)) exist, else create
    cat_govt=URIRef(bhcatres+'21')
    fetch_cat_broader(cat_govt,cat1_id)
    
    #check if cat2_id already exists, else create
    cat2_id=fetch_n_cat(cat2)
    
    #check if ((cat1 broader cat2)), ((cat2 broaderOf cat2)) exist, else create
    fetch_cat_broader(cat1_id,cat2_id)
    
    #check if org1_id already exists, else create
    ##clean and return list
    org1_ls=org_scrub(org1)
    if len(org1_ls)==1:
        org_nam=org1_ls[0]
#     elif len(org1_ls)==2:
#         org_nam=org1_ls[0]
#         org_url=org1_ls[1]
    elif len(org1_ls)==2:
        org_nam=org1_ls[0]
        if org1_ls[1].find('http')>0:
            org_url=org1_ls[1]
        else:
            org_abb=org1_ls[1]
    else:
        org_nam=org1_ls[0]
        org_url=org1_ls[1]
        org_abb=org1_ls[2]
        
    org1_id=fetch_n_org(org_nam,org_url,org_abb)
    
    #create tuple org1, org:classification, cat2_id
    g1.add((org1_id,org.classification,cat2_id)) 
    #check if org2_id already exists, else create
    org2_ls=org_scrub(org2)
    if (len(org2_ls))>0: 
        if len(org2_ls)==1:
            org_nam=org2_ls[0]
#         elif len(org2_ls)==2:
#             org_nam=org2_ls[0]
#             org_url=org2_ls[1]
        elif len(org2_ls)==2:
            org_nam=org2_ls[0]
            if org2_ls[1].find('http')>0:
                org_url=org2_ls[1]
            else:
                org_abb=org2_ls[1] 
        else:
            org_nam=org2_ls[0]
            org_url=org2_ls[1]
            org_abb=org2_ls[2]
        org2_id=fetch_n_org(org_nam,org_url,org_abb)
    #create tuple org2, org:classificatoin, cat2_id
        g1.add((org2_id,org.classification,cat2_id))
    
        #create tuples for org1-->subOrg-->org2
        fetch_org_subOrg(org1_id,org2_id)

In [16]:
len(g1)

11108

In [17]:
g1.serialize(format='nt', destination='C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/Graph_Union_2.nt')

In [18]:
#Testing
i=0
for s,p,o in g1.triples((None, bhorg.hasName,Literal("Election Commission of India"))):
    #print(s,p,o)
    for s1,p1,o1 in g1.triples((s,org.hasSubOrganization,None)):
        for s2,p2,o2 in g1.triples((o1,bhorg.hasName,None)):
            i+=1
            print(i,":",o2)

1 : Chief Electoral Officer (CEO), Andhra Pradesh
2 : Chief Electoral Officer (CEO), Nagaland
3 : Chief Electoral Officer (CEO), Andaman and Nicobar
4 : Chief Electoral Officer (CEO), Arunachal Pradesh
5 : Chief Electoral Officer (CEO), Jharkhand
6 : Chief Electoral Officer (CEO), Assam
7 : Systematic Voters� Education and Electoral Participation (SVEEP), Election Commission of India
8 : Citizen Services Portal, Election Commission of India
9 : Chief Electoral Officer (CEO), Puducherry
10 : Chief Electoral Officer (CEO), Manipur
11 : Chief Electoral Officer (CEO), Gujarat
12 : Chief Electoral Officer (CEO), Delhi
13 : Chief Electoral Officer (CEO), Mizoram
14 : Chief Electoral Officer (CEO), Punjab
15 : Chief Electoral Officer (CEO), Sikkim
16 : Chief Electoral Officer (CEO), Chhattisgarh
17 : Chief Electoral Officer (CEO), Daman and Diu
18 : National Electoral Search, Election Commission of India
19 : Chief Electoral Officer (CEO), Himachal Pradesh, india
20 : Chief Electoral Officer 

In [19]:
for s,p,o in g1.triples((None,bhorg.hasName,None)):
    print(o + "|") 

Kendriya Vidyalaya OLF, Raipur, Dehradun|
Kendriya Vidyalaya, Baran, Rajasthan|
National Institute of Animal Biotechnology (NIAB), Hyderabad, Andhra Pradesh|
Indian Rubber Manufactures Research Association (IRMRA), Thane|
School of Technology and Computer Science, Tata Institute of Fundamental Research (TIFR), Mumbai|
Institute of Microbial Technology (IMTECH) , Chandigarh|
Kendriya Vidyalaya No.5 Bathinda Cantt, Punjab|
Tourism|
Bioinformatics Infrastructure Facility (BIF), College of Veterinary Science, Assam Agricultural University|
Kendriya Vidyalaya Ballygunge, Kolkata|
Madras Institute of Development Studies (MIDS)|
Department of Earth Sciences, IIT Bombay|
Atomic Energy Education Society (AEES)|
Kendriya Vidyalaya Umaria, Madhya Pradesh|
Homi Bhabha Centre for Science Education (HBCSE)|
Kendriya Vidyalaya, JNU Campus, New Delhi|
National Centre for Earth Science Studies (NCESS)|
Regional Telecom Training centre (RTTC), Ahmedabad, Gujarat|
Handlooms and Textiles Directorate|
Kend

Central Glass and Ceramic Research Institute|
Presidents Secretariat Helpline|
Kendriya Vidyalaya No 2, Delhi Cantonment, New Delhi|
Department of Land Resources (DLR)|
Ministry of Mines|
NITI Aayog/Planning Commission|
North Eastern Institute of Ayurveda and Homoeopathy (NEIAH), Shillong, Meghalaya|
Ministry of Agriculture and Farmers Welfare|
Kendriya Vidyalaya No.2 Nausenabagh, Visakhapatnam|
Chief Electoral Officer (CEO), Himachal Pradesh, india|
e-Learning Courses of National Institute of Public Cooperation and Child Development (NIPCCD)|
Accountant General, Nagaland|
Sardar Vallabhbhai Patel National Police Academy|
Institute of Hotel Management (IHM), Hajipur, Patna, Bihar|
Regional Telecom Training Centre (RTTC), Hydrebad|
Indian Institute of Chemical Biology (IICB)|
Institute of Hotel Management, Catering Technology and Applied Nutrition, Mumbai|
National Skill Trainers Institute (NSTI) for Women, Dakopgre Tura, West Garo Hills, Meghalaya (formerly Regional Vocational Training